<a href="https://colab.research.google.com/github/e3la/instagram2digitalcommons/blob/main/instagram2digitalcommons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This first block of code asks if you want to upload your zip or if you want to grab it off google drive. While developing this tool I kept having google colab forget about my zip file and I realized I couldn't keep spending so much time waiting for it to re-upload while I developed it, and thus, this choice was born.

In [3]:
import os
import shutil # For file operations like deleting directories
from google.colab import files, drive
import zipfile

zip_filepath = None # This will store the path to the zip file
uploaded_filename_original = None # To store the original name from upload

# --- Helper function to clean up previous uploads from /content/ if method 1 is chosen ---
def cleanup_content_directory(filename_to_keep=None):
    """Removes all files and folders from /content/ except specified ones."""
    print("Cleaning up /content/ directory...")
    items_to_preserve = ["drive", "sample_data"] # Default Colab folders
    if filename_to_keep:
        items_to_preserve.append(os.path.basename(filename_to_keep))

    for item in os.listdir("/content/"):
        if item in items_to_preserve:
            continue
        item_path = os.path.join("/content/", item)
        try:
            if os.path.isfile(item_path) or os.path.islink(item_path):
                os.unlink(item_path)
                # print(f"Removed file: {item_path}")
            elif os.path.isdir(item_path):
                shutil.rmtree(item_path)
                # print(f"Removed directory: {item_path}")
        except Exception as e:
            print(f"Failed to delete {item_path}. Reason: {e}")
    print("Cleanup of /content/ complete.")

# --- Ask the user how they want to provide the file ---
while True:
    print("-" * 50)
    method = input(
        "How do you want to provide the ZIP file?\n"
        "1. Upload directly to Colab (for smaller files, keeps original name).\n"
        "2. Use the ZIP file from Google Drive (searches 'MyDrive/i2dc/' for a unique .zip file).\n"
        "Enter choice (1 or 2): "
    ).strip()
    if method in ['1', '2']:
        break
    else:
        print("Invalid choice. Please enter 1 or 2.")
print("-" * 50)

# --- Option 1: Direct Upload ---
if method == '1':
    print("Selected: Upload directly to Colab.")
    print("Please wait for the upload dialog and select your ZIP file...")

    # Clean up /content/ before new upload
    cleanup_content_directory()

    try:
        uploaded = files.upload()
        if not uploaded:
            print("No file was uploaded. Exiting.")
        else:
            # Get the uploaded file name (key in the 'uploaded' dict)
            uploaded_filename_original = list(uploaded.keys())[0]

            if not uploaded_filename_original.lower().endswith('.zip'):
                print(f"Error: The uploaded file '{uploaded_filename_original}' is not a ZIP file.")
                # Clean up the wrongly uploaded file
                wrong_file_path = os.path.join("/content/", uploaded_filename_original)
                if os.path.exists(wrong_file_path):
                    os.remove(wrong_file_path)
                uploaded_filename_original = None # Reset as it's not a valid zip
            else:
                # The file is uploaded directly to /content/
                zip_filepath = os.path.join("/content/", uploaded_filename_original)
                print(f"Successfully uploaded: '{uploaded_filename_original}'")
                print(f"File path in Colab: {zip_filepath}")

    except Exception as e:
        print(f"An error occurred during upload: {e}")
        print("If the file is too large, please try the Google Drive option next time.")

# --- Option 2: Google Drive ---
elif method == '2':
    print("Selected: Use ZIP file from Google Drive.")
    print("Attempting to mount Google Drive...")
    try:
        drive.mount('/content/drive', force_remount=True)
        print("Google Drive mounted successfully at /content/drive")

        gdrive_my_drive_path = "/content/drive/MyDrive/"
        target_folder_name = "i2dc"
        target_folder_path_in_drive = os.path.join(gdrive_my_drive_path, target_folder_name)

        print(f"Searching for a unique .zip file in: '{target_folder_path_in_drive}'")

        if not os.path.isdir(target_folder_path_in_drive):
            print(f"Error: The folder '{target_folder_path_in_drive}' ('{target_folder_name}' in your MyDrive) does not exist.")
            print(f"Please ensure you have a folder named '{target_folder_name}' directly under 'My Drive' containing your .zip file.")
        else:
            zip_files_found = []
            for item_name in os.listdir(target_folder_path_in_drive):
                item_full_path = os.path.join(target_folder_path_in_drive, item_name)
                if os.path.isfile(item_full_path) and item_name.lower().endswith('.zip'):
                    zip_files_found.append(item_full_path)

            if len(zip_files_found) == 0:
                print(f"No .zip files found in '{target_folder_path_in_drive}'.")
            elif len(zip_files_found) == 1:
                zip_filepath = zip_files_found[0]
                uploaded_filename_original = os.path.basename(zip_filepath) # Get original name from path
                print(f"Found ZIP file: '{uploaded_filename_original}'")
                print(f"File path in Colab (via Drive): {zip_filepath}")
            else:
                print(f"Error: Multiple .zip files found in '{target_folder_path_in_drive}':")
                for f_path in zip_files_found:
                    print(f" - {os.path.basename(f_path)}")
                print("Please ensure there is only one .zip file in that folder for this option to work automatically.")

    except Exception as e:
        print(f"An error occurred while mounting or accessing Google Drive: {e}")

print("-" * 50)

# --- Proceed with the zip file if one was successfully identified ---
if zip_filepath and os.path.exists(zip_filepath):
    print(f"\nProceeding with ZIP file: '{uploaded_filename_original}'")
    print(f"Located at: {zip_filepath}")

    # --- Your next steps using zip_filepath ---
    extract_to_folder = "/content/extracted_data" # Define your extraction path

    # Clean up previous extraction if it exists
    if os.path.exists(extract_to_folder):
        print(f"Cleaning up previous extraction at '{extract_to_folder}'...")
        shutil.rmtree(extract_to_folder)
    os.makedirs(extract_to_folder, exist_ok=True)
    print(f"Extraction target directory: '{extract_to_folder}'")

    try:
        print(f"Attempting to extract '{uploaded_filename_original}'...")
        with zipfile.ZipFile(zip_filepath, 'r') as zip_ref:
            zip_ref.extractall(extract_to_folder)
        print(f"Successfully extracted '{uploaded_filename_original}' to '{extract_to_folder}'")

        print("\nContents of the extracted folder:")
        extracted_items = os.listdir(extract_to_folder)
        if not extracted_items:
            print("(The extracted folder is empty)")
        else:
            for item in extracted_items:
                print(f"- {item}")

    except zipfile.BadZipFile:
        print(f"Error: The file '{uploaded_filename_original}' is not a valid ZIP file or is corrupted.")
    except Exception as e:
        print(f"An error occurred during extraction: {e}")

elif method == '1' and not zip_filepath and uploaded_filename_original is None and 'uploaded' in locals() and not uploaded:
    # This case is already handled by "No file was uploaded."
    pass
elif method == '1' and not zip_filepath and uploaded_filename_original is None:
    # This case is for "uploaded file was not a ZIP"
    print("\nCannot proceed as the uploaded file was not a valid ZIP file.")
elif not zip_filepath:
    print("\nNo ZIP file was successfully specified or found. Cannot proceed with further operations.")
elif not os.path.exists(zip_filepath): # Should be rare if logic above is correct
     print(f"\nError: The determined ZIP file path '{zip_filepath}' does not seem to exist. This is unexpected. Cannot proceed.")

print("-" * 50)
print("Script finished.")

--------------------------------------------------
How do you want to provide the ZIP file?
1. Upload directly to Colab (for smaller files, keeps original name).
2. Use the ZIP file from Google Drive (searches 'MyDrive/i2dc/' for a unique .zip file).
Enter choice (1 or 2): 2
--------------------------------------------------
Selected: Use ZIP file from Google Drive.
Attempting to mount Google Drive...
Mounted at /content/drive
Google Drive mounted successfully at /content/drive
Searching for a unique .zip file in: '/content/drive/MyDrive/i2dc'
Found ZIP file: 'instagram-umsllibraries-2025-03-07-Ardjbhx1.zip'
File path in Colab (via Drive): /content/drive/MyDrive/i2dc/instagram-umsllibraries-2025-03-07-Ardjbhx1.zip
--------------------------------------------------

Proceeding with ZIP file: 'instagram-umsllibraries-2025-03-07-Ardjbhx1.zip'
Located at: /content/drive/MyDrive/i2dc/instagram-umsllibraries-2025-03-07-Ardjbhx1.zip
Extraction target directory: '/content/extracted_data'
Attem

In [61]:
# 🔍 Revised: Inspect contents of the Instagram archive using your file counting logic
import os

MEDIA_DIR = "/content/extracted_data"

print(f"\n🔎 Scanning '{MEDIA_DIR}' for Instagram data...")

file_count = 0
image_count = 0
video_count = 0

for root, dirs, files in os.walk(MEDIA_DIR):
    for file in files:
        file_count += 1
        if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.webp')):
            image_count += 1
        elif file.lower().endswith('.mp4'):
            video_count += 1

print("\n📊 Archive Summary")
print("-----------------------------------------")
print(f"Total files: {file_count}")
print(f"Image files: {image_count}")
print(f"MP4 video files: {video_count}")
print("You can also browse these files using the 'Files' panel on the left sidebar.")
print("-----------------------------------------")

# 📦 Count Instagram media types from extracted JSON data
import json

# Base directory: previously defined as extract_dir
base_dir = MEDIA_DIR
activity_dir = None

# Base directory: previously defined as extract_dir
base_dir = MEDIA_DIR

# Default activity folder name
activity_dir = 'your_instagram_activity'
print(f"📁 Using default Instagram activity folder: '{activity_dir}'")

media_json_path = os.path.join(base_dir, activity_dir, 'media')
print(f"\n📂 Searching media JSON files in: {media_json_path}")
# Define target files
json_files = {
    "posts": "posts_1.json",
    "reels": "reels.json",
    "stories": "stories.json"
}

counts = {}

for media_type, filename in json_files.items():
    file_path = os.path.join(media_json_path, filename)
    print(f"\n🔍 {media_type.capitalize()}: {filename}")

    try:
        if not os.path.exists(file_path):
            print("   ⚠️ File not found.")
            counts[media_type] = 0
            continue

        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        if isinstance(data, list):  # posts_1.json
            counts[media_type] = len(data)
            print(f"   ✅ Found {counts[media_type]} entries (flat list).")

        elif isinstance(data, dict):
            top_key = {
                "reels": "ig_reels_media",
                "stories": "ig_stories"
            }.get(media_type)

            if top_key not in data:
                print(f"   ⚠️ Key '{top_key}' not found.")
                counts[media_type] = 0
                continue

            flat_list = []
            for group in data[top_key]:
                if isinstance(group, dict) and "media" in group:
                    flat_list.extend(group["media"])
                else:
                    flat_list.append(group)

            counts[media_type] = len(flat_list)
            print(f"   ✅ Found {counts[media_type]} entries (flattened).")

        else:
            print("   ❌ Unrecognized JSON structure.")
            counts[media_type] = 0

    except json.JSONDecodeError:
        print("   ❌ Error decoding JSON.")
        counts[media_type] = 0
    except Exception as e:
        print(f"   ❌ Unexpected error: {e}")
        counts[media_type] = 0

# prompt: this is in the json
# {
#   "profile_user": [
#     {
...
#         "Username": {
#           "href": "",
#           "value": "umsllibraries",
#           "timestamp": 0
# in the file /content/extracted_data/personal_information/personal_information/personal_information.json
# and I want to pull out just the Username value

# Path to the JSON file
json_file_path = "/content/extracted_data/personal_information/personal_information/personal_information.json"

# Load the JSON data
try:
    with open(json_file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # Navigate through the structure to find the Username value
    username_value = None
    if isinstance(data, dict) and "profile_user" in data and isinstance(data["profile_user"], list) and len(data["profile_user"]) > 0:
        # Assuming the relevant data is in the first element of the profile_user list
        profile_info = data["profile_user"][0]
        if isinstance(profile_info, dict) and "string_map_data" in profile_info and isinstance(profile_info["string_map_data"], dict):
            string_data = profile_info["string_map_data"]
            if "Username" in string_data and isinstance(string_data["Username"], dict) and "value" in string_data["Username"]:
                username_value = string_data["Username"]["value"]

    # Print the extracted username
    if username_value is not None:
        print(f"\nExtracted Username: {username_value}")
    else:
        print("Username not found in the expected structure.")

except FileNotFoundError:
    print(f"Error: The file was not found at {json_file_path}")
except json.JSONDecodeError:
    print(f"Error: Could not decode JSON from {json_file_path}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")



🔎 Scanning '/content/extracted_data' for Instagram data...

📊 Archive Summary
-----------------------------------------
Total files: 1549
Image files: 1229
MP4 video files: 178
You can also browse these files using the 'Files' panel on the left sidebar.
-----------------------------------------
📁 Using default Instagram activity folder: 'your_instagram_activity'

📂 Searching media JSON files in: /content/extracted_data/your_instagram_activity/media

🔍 Posts: posts_1.json
   ✅ Found 646 entries (flat list).

🔍 Reels: reels.json
   ✅ Found 25 entries (flattened).

🔍 Stories: stories.json
   ✅ Found 485 entries (flattened).

Extracted Username: umsllibraries


In [62]:
# CELL TO RUN BEFORE CONTENT PROCESSING CELLS
# This cell will iterate through all .json files in the media directory,
# back them up, and attempt to fix mojibake in known text fields using ftfy.

# 1. Install ftfy
!pip install ftfy -q

import os
import json
import shutil # For copying files
import ftfy

print("--- Starting JSON fixing process for all media files ---")

# Define the path to the media directory
MEDIA_DIR_BASE = "/content/extracted_data" # Make sure this matches your setup
TARGET_MEDIA_DIR = os.path.join(MEDIA_DIR_BASE, 'your_instagram_activity', 'media')

if not os.path.isdir(TARGET_MEDIA_DIR):
    print(f"❌ ERROR: Media directory not found at: {TARGET_MEDIA_DIR}")
    print("Please ensure the extraction in the first cell was successful and the path is correct.")
else:
    print(f"✅ Found media directory: {TARGET_MEDIA_DIR}")

    total_files_processed = 0
    total_files_changed_by_ftfy = 0
    total_titles_fixed_overall = 0

    # Helper function to fix text fields in common Instagram JSON structures
    def fix_text_fields_in_data(loaded_data, filename_basename):
        """
        Modifies loaded_data in-place by applying ftfy.fix_text to known text fields.
        Returns: (bool: data_was_changed, int: titles_fixed_count)
        """
        data_was_changed_locally = False
        titles_fixed_count_locally = 0

        def _apply_fix(obj, key):
            nonlocal data_was_changed_locally, titles_fixed_count_locally
            original_text = obj.get(key)
            if isinstance(original_text, str) and original_text: # Only fix non-empty strings
                fixed_text = ftfy.fix_text(original_text)
                if fixed_text != original_text:
                    obj[key] = fixed_text
                    data_was_changed_locally = True
                    titles_fixed_count_locally += 1

        if filename_basename == "reels.json" or filename_basename == "stories.json":
            top_level_key = "ig_reels_media" if filename_basename == "reels.json" else "ig_stories"
            if isinstance(loaded_data, dict) and top_level_key in loaded_data:
                item_list = loaded_data.get(top_level_key, [])
                for item_dict in item_list: # Can be group or direct media item
                    if isinstance(item_dict, dict):
                        if "media" in item_dict and isinstance(item_dict["media"], list): # It's a group
                            for media_sub_item_dict in item_dict["media"]:
                                if isinstance(media_sub_item_dict, dict):
                                    _apply_fix(media_sub_item_dict, "title")
                        else: # It's a direct media item in the list
                            _apply_fix(item_dict, "title")
            else:
                print(f"    ⚠️ Structure of {filename_basename} not as expected (missing '{top_level_key}' or not a dict). Skipping detailed fix.")


        elif filename_basename == "posts_1.json":
            if isinstance(loaded_data, list): # posts_1.json is a list of post objects
                for post_item_dict in loaded_data:
                    if isinstance(post_item_dict, dict):
                        _apply_fix(post_item_dict, "title") # Overall post caption/title

                        media_list_in_post = post_item_dict.get("media", [])
                        if isinstance(media_list_in_post, list):
                            for media_item_dict in media_list_in_post:
                                if isinstance(media_item_dict, dict):
                                    _apply_fix(media_item_dict, "title") # Caption for individual media
            else:
                print(f"    ⚠️ Structure of {filename_basename} not as expected (not a list). Skipping detailed fix.")

        # Add handlers for other specific JSON files here if needed
        # else:
        #     print(f"    ℹ️ No specific ftfy fixing logic defined for {filename_basename}. Text fields might not be fixed.")

        return data_was_changed_locally, titles_fixed_count_locally

    # Iterate through all files in the target directory
    for filename in os.listdir(TARGET_MEDIA_DIR):
        if filename.lower().endswith('.json'):
            total_files_processed += 1
            current_json_path = os.path.join(TARGET_MEDIA_DIR, filename)
            print(f"\n--- Processing: {filename} ---")

            # 2. Backup original file
            backup_json_path = os.path.join(TARGET_MEDIA_DIR, f"{os.path.splitext(filename)[0]}-original.json")
            if not os.path.exists(backup_json_path):
                try:
                    shutil.copy2(current_json_path, backup_json_path)
                    print(f"  💾 Original backed up to: {os.path.basename(backup_json_path)}")
                except Exception as e:
                    print(f"  ⚠️ WARNING: Could not create backup of {filename}: {e}")
            else:
                print(f"  ℹ️ Backup '{os.path.basename(backup_json_path)}' already exists. Not overwriting.")

            # 3. Load, Fix, and Save the current JSON file
            try:
                print(f"  L_oading {filename} for fixing...")
                with open(current_json_path, 'r', encoding='utf-8') as f:
                    json_content = json.load(f)
                print(f"    {filename} loaded successfully.")

                # Apply ftfy fixes based on file type
                data_actually_changed_in_file, titles_fixed_in_file = fix_text_fields_in_data(json_content, filename)

                if titles_fixed_in_file > 0:
                    print(f"    🔧 Titles/text fields fixed by ftfy in {filename}: {titles_fixed_in_file}")
                    total_titles_fixed_overall += titles_fixed_in_file

                if data_actually_changed_in_file:
                    total_files_changed_by_ftfy +=1
                    print(f"  S_aving modified {filename} back to disk...")
                    try:
                        with open(current_json_path, 'w', encoding='utf-8') as f:
                            json.dump(json_content, f, ensure_ascii=False, indent=2)
                        print(f"    ✅ Successfully saved fixed {filename}.")
                    except Exception as e:
                        print(f"    ❌ ERROR saving modified {filename}: {e}")
                elif titles_fixed_in_file > 0 and not data_actually_changed_in_file : # Should not happen if titles_fixed_in_file > 0
                    print(f"    ℹ️ ftfy processed text in {filename} but resulted in no net change. File not rewritten.")
                else: # No titles fixed or no changes made
                    print(f"    ℹ️ No changes made by ftfy to {filename}. File not rewritten.")

            except json.JSONDecodeError:
                print(f"  ❌ ERROR: Could not decode JSON from {filename}. The file might be corrupted. Skipping this file.")
            except FileNotFoundError:
                 print(f"  ❌ ERROR: File {filename} not found during processing loop (should not happen if listed). Skipping.")
            except Exception as e:
                print(f"  ❌ An unexpected error occurred while processing {filename}: {e}")

    print("\n--- Summary of JSON fixing process ---")
    print(f"Total .json files found and processed: {total_files_processed}")
    print(f"Total files modified by ftfy and resaved: {total_files_changed_by_ftfy}")
    print(f"Total text fields fixed across all files: {total_titles_fixed_overall}")
    print("--- Finished JSON fixing process for all media files ---")

--- Starting JSON fixing process for all media files ---
✅ Found media directory: /content/extracted_data/your_instagram_activity/media

--- Processing: posts_1.json ---
  ℹ️ Backup 'posts_1-original.json' already exists. Not overwriting.
  L_oading posts_1.json for fixing...
    posts_1.json loaded successfully.
    ℹ️ No changes made by ftfy to posts_1.json. File not rewritten.

--- Processing: reels-original.json ---
  ℹ️ Backup 'reels-original-original.json' already exists. Not overwriting.
  L_oading reels-original.json for fixing...
    reels-original.json loaded successfully.
    ℹ️ No changes made by ftfy to reels-original.json. File not rewritten.

--- Processing: stories.json ---
  ℹ️ Backup 'stories-original.json' already exists. Not overwriting.
  L_oading stories.json for fixing...
    stories.json loaded successfully.
    ℹ️ No changes made by ftfy to stories.json. File not rewritten.

--- Processing: reels.json ---
  ℹ️ Backup 'reels-original.json' already exists. Not ov

In [63]:
import os
import shutil
from google.colab import files, drive
import zipfile
import json
from datetime import datetime
import pandas as pd
from shutil import copy2
import re # For hashtag extraction

# Ensure MEDIA_DIR is defined, as it's used in subsequent cells
MEDIA_DIR = "/content/extracted_data" # This should be consistent with the extraction path

media_type = 'reels' # We are focusing on reels
output_dir = os.path.join(MEDIA_DIR, f'{media_type}_export_dc_format') # New output dir name
os.makedirs(output_dir, exist_ok=True)

# Excel and ZIP file paths
excel_filename = f'{media_type}_metadata_dc_format.xlsx'
excel_path = os.path.join(output_dir, excel_filename)
readme_path = os.path.join(output_dir, 'README.txt')
zip_path = os.path.join(output_dir, f'{media_type}_package_dc_format.zip') # New zip name

# Attempt to get username_value from global scope (set in a previous cell)
instagram_handle = globals().get('username_value', "unknown_user")
print(f"Using Instagram handle: {instagram_handle}")

# --- Helper function to extract hashtags ---
def extract_hashtags(text):
    if not isinstance(text, str):
        return ""
    hashtags = re.findall(r"#(\w+)", text)
    return ", ".join(hashtags)

# --- Load Reels JSON Data ---
instagram_media_data = {}
try:
    reels_json_path = os.path.join(MEDIA_DIR, 'your_instagram_activity', 'media', 'reels.json')
    if os.path.exists(reels_json_path):
         with open(reels_json_path, 'r', encoding='utf-8') as f:
            reels_full_data = json.load(f)
            if isinstance(reels_full_data, dict) and "ig_reels_media" in reels_full_data:
                 flat_list = []
                 for group in reels_full_data["ig_reels_media"]:
                     if isinstance(group, dict) and "media" in group:
                         flat_list.extend(group["media"])
                     else:
                         flat_list.append(group)
                 instagram_media_data['reels'] = flat_list # Assign to 'reels' key
            else:
                print("Warning: Reels JSON structure unexpected or 'ig_reels_media' key missing.")
                instagram_media_data['reels'] = []
    else:
        print(f"Reels JSON file not found at: {reels_json_path}")
        instagram_media_data['reels'] = []
except Exception as e:
    print(f"Error loading Reels JSON for export: {e}")
    instagram_media_data['reels'] = []

# --- Process Reels Data ---
reels_data_to_process = instagram_media_data.get('reels') # Get data from 'reels' key

if not reels_data_to_process:
    print("No Reels data loaded or found for export.")
else:
    print(f"Preparing to export metadata for {len(reels_data_to_process)} Reels in Digital Commons format...")

    excel_data_rows = []

    # Define the new column order for the Excel sheet
    column_names = [
        'title', 'fulltext_url', 'additional_files', 'keywords', 'abstract',
        'author1_fname', 'author1_mname', 'author1_lname', 'author1_suffix',
        'author1_email', 'author1_institution', 'author1_is_corporate',
        'author2_fname', 'author2_mname', 'author2_lname', 'author2_suffix',
        'author2_email', 'author2_institution', 'author2_is_corporate',
        'author3_fname', 'author3_mname', 'author3_lname', 'author3_suffix',
        'author3_email', 'author3_institution', 'author3_is_corporate',
        'author4_fname', 'author4_mname', 'author4_lname', 'author4_suffix',
        'author4_email', 'author4_institution', 'author4_is_corporate',
        'disciplines', 'instagram_username', 'document_type'
    ]

    copied_files_for_zip = []

    for i, item in enumerate(reels_data_to_process):
        original_reel_uri = item.get('uri')
        if not original_reel_uri:
            print(f"Warning: Reel item {i} has no URI, skipping.")
            continue

        media_path = os.path.join(MEDIA_DIR, original_reel_uri)
        if not os.path.exists(media_path):
            print(f"Warning: Media file not found - {media_path}, skipping reel item.")
            continue

        # --- Date and Filename (Reel Video) ---
        timestamp = item.get('creation_timestamp')
        date_obj = datetime.fromtimestamp(timestamp) if timestamp else datetime.now()
        date_str_for_title = date_obj.strftime('%Y-%m-%d')

        ext = os.path.splitext(original_reel_uri)[-1]
        handle_for_filename = instagram_handle if instagram_handle else "unknown_user"
        original_file_basename = os.path.splitext(os.path.basename(original_reel_uri))[0]
        sanitized_original_basename = ''.join(c if c.isalnum() else '_' for c in original_file_basename)

        # This is the reel video filename, goes into 'fulltext_url'
        reel_export_filename = f"instagram_{handle_for_filename}_reel_{date_str_for_title}_{i+1}_{sanitized_original_basename}{ext}"
        reel_export_path = os.path.join(output_dir, reel_export_filename)

        try:
            copy2(media_path, reel_export_path)
            copied_files_for_zip.append(reel_export_path)
        except Exception as e:
            print(f"Error copying media file {media_path} to {reel_export_path}: {e}. Skipping this reel.")
            continue

        # --- Abstract and Keywords ---
        original_caption = item.get('title', '') # This is the Instagram caption
        keywords_str = extract_hashtags(original_caption)

        # --- SRT File (Additional File) ---
        srt_export_filename = '' # Initialize to empty
        original_srt_uri = ''

        try:
            subtitles_data = item.get('media_metadata', {}).get('video_metadata', {}).get('subtitles', {})
            original_srt_uri = subtitles_data.get('uri', '')

            if original_srt_uri:
                srt_original_full_path = os.path.join(MEDIA_DIR, original_srt_uri)
                if os.path.exists(srt_original_full_path):
                    base_export_videoname, _ = os.path.splitext(reel_export_filename)
                    srt_export_filename = base_export_videoname + ".srt" # This goes into 'additional_files'
                    srt_export_path = os.path.join(output_dir, srt_export_filename)

                    copy2(srt_original_full_path, srt_export_path)
                    copied_files_for_zip.append(srt_export_path)
                else:
                    # srt_export_filename remains empty if source SRT not found
                    pass
        except (KeyError, TypeError):
            # srt_export_filename remains empty
            pass

        # --- Construct new title ---
        dc_title = f"Instagram {handle_for_filename} {date_str_for_title}"

        # --- Prepare row data for Excel ---
        row = {
            'title': dc_title,
            'fulltext_url': reel_export_filename,
            'additional_files': srt_export_filename, # Will be empty if no SRT
            'keywords': keywords_str,
            'abstract': original_caption,
            'author1_fname': '', 'author1_mname': '', 'author1_lname': '', 'author1_suffix': '',
            'author1_email': '', 'author1_institution': '', 'author1_is_corporate': '',
            'author2_fname': '', 'author2_mname': '', 'author2_lname': '', 'author2_suffix': '',
            'author2_email': '', 'author2_institution': '', 'author2_is_corporate': '',
            'author3_fname': '', 'author3_mname': '', 'author3_lname': '', 'author3_suffix': '',
            'author3_email': '', 'author3_institution': '', 'author3_is_corporate': '',
            'author4_fname': '', 'author4_mname': '', 'author4_lname': '', 'author4_suffix': '',
            'author4_email': '', 'author4_institution': '', 'author4_is_corporate': '',
            'disciplines': '',
            'instagram_username': instagram_handle,
            'document_type': 'Instagram Reel'
        }
        excel_data_rows.append(row)

    # --- Create DataFrame and save to Excel ---
    if excel_data_rows:
        df = pd.DataFrame(excel_data_rows, columns=column_names)
        try:
            df.to_excel(excel_path, index=False, engine='openpyxl')
            print(f"Metadata written to Excel: {excel_path}")
        except Exception as e:
            print(f"Error writing to Excel file {excel_path}: {e}")
            print("Make sure 'openpyxl' library is installed (pip install openpyxl).")
    else:
        print("No data to write to Excel.")

    # --- Write README file (Updated) ---
    with open(readme_path, 'w', encoding='utf-8') as f:
        f.write(f"""Instagram Reels Export Package (Digital Commons Format)
=====================================================

Handle: @{instagram_handle}
Exported: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

This package contains:
- Exported Reel video files.
- Associated .srt subtitle files (if they existed in the archive and were found), renamed to match their corresponding video files.
- An Excel file ({excel_filename}) with metadata for each reel, formatted for potential Digital Commons import.
- This README.txt file.

Fields in the Excel file ({excel_filename}):
- title: A generated title in the format "Instagram [username] [YYYY-MM-DD]".
- fulltext_url: The filename of the exported reel video file in this package. This file should be uploaded as the primary file.
- additional_files: The filename of the exported .srt subtitle file (if available) in this package. This should be uploaded as an additional file.
- keywords: Comma-separated hashtags extracted from the reel's original caption.
- abstract: The original caption/text of the Instagram reel.
- author1_fname to author4_is_corporate: Fields for author information (currently left blank).
- disciplines: Field for academic disciplines (currently left blank).
- instagram_username: The Instagram handle from which the reel originated.
- document_type: Set to "Instagram Reel".

Generated by Instagram archive processing script.
""")
    print(f"README file written to: {readme_path}")

    # --- Zip everything up ---
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        if os.path.exists(excel_path):
            zipf.write(excel_path, arcname=os.path.basename(excel_path))
        zipf.write(readme_path, arcname='README.txt')
        for file_to_zip in copied_files_for_zip: # Includes videos and SRTs
            if os.path.exists(file_to_zip):
                zipf.write(file_to_zip, arcname=os.path.basename(file_to_zip))
            else:
                print(f"Warning: File {file_to_zip} not found for zipping.")
    print(f"ZIP archive created: {zip_path}")

# The last cell (for downloading) will need to be updated to point to this new zip_path
# e.g., source_zip_path = "/content/extracted_data/reels_export_dc_format/reels_package_dc_format.zip"
# and target_zip_filename = "reels_dc_format.zip"

Using Instagram handle: umsllibraries
Preparing to export metadata for 25 Reels in Digital Commons format...
Metadata written to Excel: /content/extracted_data/reels_export_dc_format/reels_metadata_dc_format.xlsx
README file written to: /content/extracted_data/reels_export_dc_format/README.txt
ZIP archive created: /content/extracted_data/reels_export_dc_format/reels_package_dc_format.zip


In [64]:
import os
import shutil
from google.colab import files # Make sure this is imported

# --- Assuming your previous script has run and created the source ZIP ---
# Define the source and target paths
source_zip_path = "/content/extracted_data/reels_export_dc_format/reels_package_dc_format.zip"
target_base_dir = "/content/batchup" # The directory where you want the new ZIP
target_zip_filename = "reels.zip"     # The desired name for the ZIP in the target directory
target_zip_path = os.path.join(target_base_dir, target_zip_filename)

# 1. Check if the source ZIP file exists
if not os.path.exists(source_zip_path):
    print(f"❌ ERROR: Source ZIP file not found at: {source_zip_path}")
    print("Please ensure the previous steps to create the ZIP were successful.")
else:
    print(f"✔️ Source ZIP found: {source_zip_path}")

    # 2. Ensure the target directory exists, create it if not
    os.makedirs(target_base_dir, exist_ok=True)
    print(f"✔️ Ensured target directory exists: {target_base_dir}")

    try:
        # 3. Copy the file
        shutil.copy2(source_zip_path, target_zip_path) # copy2 preserves metadata
        print(f"✅ Successfully copied '{os.path.basename(source_zip_path)}' to '{target_zip_path}'")

        # 4. Offer a download link for the new file
        print(f"\n⬇️ Click the link below to download '{target_zip_filename}':")
        # Note: files.download() directly initiates the download in the browser
        # It doesn't print a clickable link in the classic HTML sense in the output cell,
        # but Colab's UI will typically show a download prompt or progress.
        files.download(target_zip_path)
        print(f"(If download doesn't start automatically, check your browser's download manager or pop-up blocker.)")
        print(f"The file is located at: {target_zip_path} in the Colab environment.")

    except Exception as e:
        print(f"❌ ERROR during copy or download: {e}")

# Example of how you might integrate this at the end of your existing script:
# ... (your existing script that creates /content/extracted_data/reels_export/reels_package.zip) ...
# print(f"ZIP archive created: {zip_path}") # zip_path from your previous script would be source_zip_path here

# --- Add the copy and download logic here ---
# (The code block above would go here, making sure source_zip_path matches
# the zip_path variable from your previous script part if you used a variable)

✔️ Source ZIP found: /content/extracted_data/reels_export_dc_format/reels_package_dc_format.zip
✔️ Ensured target directory exists: /content/batchup
✅ Successfully copied 'reels_package_dc_format.zip' to '/content/batchup/reels.zip'

⬇️ Click the link below to download 'reels.zip':


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

(If download doesn't start automatically, check your browser's download manager or pop-up blocker.)
The file is located at: /content/batchup/reels.zip in the Colab environment.
